SD TSIA 211 TP1

Billy Nicolas
Beitz Enguerrand

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Loading data

data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('data_center_data_matrix.npy', allow_pickle=True)

# Constructing matrices for min_w ||A w - b||_2**2

matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

A = np.hstack([M, np.ones((M.shape[0],1)), -(M.T * COP_train[:,3]).T])
b = COP_train[:,3]

# Constructing matrices for the test set

M_test = (data_matrix_test - matrix_mean) / matrix_std
A_test = np.hstack([M_test, np.ones((M_test.shape[0],1)), -(M_test.T * COP_test[:,3]).T])
b_test = COP_test[:,3]

# Loading raw data
import pandas as pd
data = pd.read_csv('Raw_Dataset_May.csv')

def name_to_subcategory_and_details(col_name):
    if np.isreal(col_name):
        col_name = names[col_name]
    indices = np.nonzero((data['NAME'] == col_name).values)[0]
    if len(indices) > 0:
        subcategory = data['SUBCATEGORY'].iloc[[indices[0]]].values[0]
        details = data['DETAILS'].iloc[[indices[0]]].values[0]
        return subcategory, details
    else:
        print('unknown name')


* Question 3.1 

Assuming $ A \omega = b $
$ y(t)= b(t)=(A \omega )_t = \tilde{x} (t) ^T \omega _{1}  + \omega _{0} - y(t) \tilde{x}^T (t) \omega _2 $

so 

$ (1+ \tilde{x} (t) \omega _2) y(t) =  \tilde{x} (t) ^T \omega _{1}  + \omega _{0} $

so

$ y(t)= \frac{  \tilde{x} (t) ^T \omega _{1}  + \omega _{0}}{1+ \tilde{x} ^T (t) \omega _2} $

because $\omega$  and $ x $ are vectore we can swap products:

$ y(t)= \frac{\omega _{1} ^T \tilde{x} (t)   + \omega _{0}}{1+  \omega _2  ^T \tilde{x} (t)} $



* Question 3.2 

Solving with least squares

In [81]:
omega = np.linalg.lstsq(A, b, rcond = None)[0]

* Question 3.3

In [82]:
#evaluate solution on the test set
y_pred = A_test @ omega

mse_1 = np.sum((y_pred - b_test)**2)/len(b_test)

print(mse_1)

780.8984793523671


* Question 3.4

* Question 3.5

La dérivée vaut: $ A^T(A \omega -b) + \lambda \omega$

La fonction étant convexe (forme quadratique), on aura bien la solution optimale.

* Question 3.6  

In [74]:
lbd = 100

def l2_regul(A,b,lbd,omega):
    return 0.5*(np.linalg.norm(A @ omega - b)**2 +lbd*np.linalg.norm(omega)**2)

def grad_l2_regul(A,b,lbd,omega):
    return A.T @ (A @ omega - b) + lbd*omega

def grad_descent(A,b,lbd,omega_init,tau,eps,max_iter):
    omega = omega_init
    for i in range(max_iter):
        omega = omega - tau*grad_l2_regul(A,b,lbd,omega)
        if np.linalg.norm(grad_l2_regul(A,b,lbd,omega)) < eps:
            print('converged in', i, 'iterations')
            break
    return omega

# On prend le pas donné par le coefficient de Lipschitz

L = np.linalg.norm(A.T @ A+ lbd*np.eye(len(A[0])))
omega_init = np.ones(len(A[0]))
tau = 1/L
eps = 1e-7
max_iter = 10000

omega_descent = grad_descent(A,b,lbd,omega_init,tau,eps,max_iter)

y_pred_descent = A_test @ omega_descent

C:\Users\Nicolas\AppData\Local\Temp\ipykernel_20476\4276898646.py:18: RuntimeWarning: invalid value encountered in matmul
  L = np.linalg.norm(A.T @ A+ lbd*np.eye(len(A[0])))


In [75]:
print(omega_descent)

[ 0.54428802  0.34637572 -0.0494747  ...  0.14722499  0.23911253
  0.12784386]


In [79]:
mse_2 = np.sum((y_pred_descent - b_test)**2) / len(b_test) 
print(mse_2)

1883546.7412086632


* Question 4.1

$ f2 = \frac{1}{2} || A \omega - b || _2 ^2 $ 

$ g2 = \lambda ||  \omega || _1 $

$ prox _{g 2} (x) = argmin _y ( g2(y) + 0.5 ||x-y|| )$

$ grad f = A^T (A \omega -b) $

* Question 4.2



In [ ]:
from scipy.optimize import minimize

def proximal_operator(x, g, tau):
    objective = lambda y: g(y) + 1/(2*tau) * np.linalg.norm(y - x)**2
    result = minimize(objective, x)
    return result.x            

def proximal_grad_descent(A,b,lbd,omega_init,g2,tau,eps,max_iter):
    omega = omega_init
    for i in range(max_iter):
        omega = proximal_operator(omega-tau*grad_l2_regul(A,b,lbd,omega), g2, tau*lbd )
        if np.linalg.norm(grad_l2_regul(A,b,lbd,omega)) < eps:
            print('converged in', i, 'iterations')
            break
    return omega